In [ ]:
# HeatMap of Frequency of Listings
import folium
from folium.plugins import HeatMap, MarkerCluster

m = folium.Map(location=[41.9028, 12.4964], zoom_start=12)

heat_df = df[['latitude', 'longitude']]
heat_df = heat_df.dropna(axis=0, subset=['latitude', 'longitude'])
heat_data = [[row['latitude'], row['longitude']] for index, row in heat_df.iterrows()]

HeatMap(data=heat_data, radius=8, max_zoom=13).add_to(m)

m.save('out/heatmap.html')

In [ ]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

def create_marker_cluster_map(df):
    """
    Creates a marker cluster map using Folium library.

    Parameters:
    - df: pandas DataFrame
        The DataFrame containing latitude, longitude, and room type information.

    Returns:
    - m: folium.Map
        The marker cluster map.

    Example usage:
    >>> df = pd.DataFrame({'latitude': [41.9028, 41.8919, 41.8955],
                           'longitude': [12.4964, 12.5113, 12.4823],
                           'room_type': ['Entire home/apt', 'Private room', 'Shared room']})
    >>> create_marker_cluster_map(df)
    """

    # Create a folium map centered at a specific location
    m = folium.Map(location=[41.9028, 12.4964], zoom_start=12)

    # Create a marker cluster
    mc = MarkerCluster()

    # Create feature groups for different room types
    feature_group = {
        'Entire home/apt': folium.FeatureGroup(name='Entire home/apt'),
        'Private room': folium.FeatureGroup(name='Private room'),
        'Shared room': folium.FeatureGroup(name='Shared room'),
        'Hotel room': folium.FeatureGroup(name='Hotel room')
    }

    # Add markers to the marker cluster based on latitude, longitude, and room type
    for idx, row in df.iterrows():
        if not np.isnan(row['latitude']) and not np.isnan(row['longitude']):
            folium.CircleMarker(location=[row['latitude'], row['longitude']]).add_to(feature_group[row['room_type']])

    # Add the marker cluster to the map
    m.add_child(mc)

    # Add feature groups to the map
    for key in feature_group:
        feature_group[key].add_to(m)

    # Add layer control to the map
    folium.LayerControl(collapsed=False).add_to(m)

    return m

# Save the marker cluster map as an HTML file
create_marker_cluster_map(df).save('out/markercluster.html')


In [ ]:
## One Hot encoding for room_type
df = pd.get_dummies(df, columns=['room_type'], prefix='room_type')

In [ ]:
# One Hot encoding for neighbourhood_cleansed
df = pd.get_dummies(df, columns=['neighbourhood_cleansed'], prefix='neighbourhood')

In [ ]:
from category_encoders import BinaryEncoder

binary_encoder = BinaryEncoder()
binary_encoder.fit_transform(df['property_type'])
# Create a DataFrame with the new columns
new_columns = pd.DataFrame(binary_encoder.transform(df['property_type']), columns=binary_encoder.get_feature_names_out())

# Concatenate the new columns with the original dataset
df = pd.concat([df, new_columns], axis=1)
df.drop(columns=['property_type'], inplace=True)